In [1]:
import os
import re
from pymystem3 import Mystem
MYSTEM = Mystem()

In [2]:
def get_text(filename):
    f = open(filename, 'r', encoding='utf-8')
    text = f.read()
    f.close()
    return text

In [3]:
def clean(lemmas):
    for lemma in lemmas:
        if not str(lemma).isalpha():
            lemmas.remove(lemma)
    return lemmas


def get_lemmas(text):
    lemmas = MYSTEM.lemmatize(text.lower())
    return clean(lemmas)

In [4]:
def parts_of_speech(lemmas, d_A, d_APRO, d_V):
    len_A = 0
    len_APRO = 0
    len_V = 0
    ana = MYSTEM.analyze(' '.join(lemmas))
    
    for word in ana:
        if 'analysis' in word:
            try:
                gr = word['analysis'][0]['gr']
                pos = gr.split('=')[0].split(',')[0]
                if pos == 'A':
                    add_to_dict(word['text'], d_A)
                    len_A += 1
                if pos == 'APRO':
                    add_to_dict(word['text'], d_APRO)
                    len_APRO += 1
                if pos == 'V':
                    add_to_dict(word['text'], d_V)
                    len_V += 1
            except:
                pass
    return len_A, len_APRO, len_V

In [5]:
def add_to_dict(lemma, d):
    if lemma not in d:
        d[lemma] = 0
    d[lemma] += 1

In [6]:
d_A = {}
d_APRO = {}
d_V = {}

text = get_text('first_sent.txt')
len_A, len_APRO, len_V = parts_of_speech(get_lemmas(text), d_A, d_APRO, d_V)

In [7]:
from nltk import sent_tokenize

In [8]:
d2_A = {}
d2_APRO = {}
d2_V = {}

text = get_text('other_sent.txt')
sentences = sent_tokenize(text)
lt = len(sentences)
l = lt // 10

len2_A = 0
len2_APRO = 0
len2_V = 0

for i in range(1, 11):
    ln_A, ln_APRO, ln_V = parts_of_speech(get_lemmas(' '.join(sentences[(l * (i - 1)):(l * i)])), d2_A, d2_APRO, d2_V)
    len2_A += ln_A
    len2_APRO += ln_APRO
    len2_V += ln_V
    print(i)
    
if lt % 10 != 0:
    ln_A, ln_APRO, ln_V = parts_of_speech(get_lemmas(' '.join(sentences[(l * 10):])), d2_A, d2_APRO, d2_V)
    len2_A += ln_A
    len2_APRO += ln_APRO
    len2_V += ln_V

1
2
3
4
5
6
7
8
9
10


In [9]:
import pandas as pd
import re

In [10]:
def clean_text(text):
    while re.findall(r'[^а-я\.\[\]]', text):
        text = re.sub(r'[^а-я\.\[\]]', '', text)
    return text

In [11]:
def get_word(corp, d_A, d_APRO, d_V):
    for i in range(len(corp)):
        now = clean_text(corp[i].lower())
        
        while re.search('прилагательное', now):
            match = re.search(r'\[([а-я]*)\.прилагательное(\.)*\]', now)
            if match:
                add_to_dict(match.group(1), d_A)
                now = re.sub(r'прилагательное', '', now, 1)
                
        while re.search('местоимение', now):
            match = re.search(r'\[([а-я]*)\.местоимение(\.)*\]', now)
            if match:
                add_to_dict(match.group(1), d_APRO)
                now = re.sub(r'местоимение', '', now, 1)
                
        while re.search('глагол', now):
            match = re.search(r'\[([а-я]*)\.глагол(\.)*\]', now)
            if match:
                add_to_dict(match.group(1), d_V)
                now = re.sub(r'глагол', '', now, 1)  

In [12]:
def get_lemmat_text(filename, d_A, d_APRO, d_V, d2_A, d2_APRO, d2_V):
    text = get_text(filename)
    df = pd.read_csv(filename, sep='|', index_col=0, header=None)
    
    df_first = df.loc[df[1] == 1]
    first_sent = df_first[2]
    first_sent = first_sent.reset_index(drop=True)
    get_word(first_sent, d_A, d_APRO, d_V)
    
    df_other = df.loc[df[1] == 0]
    other_sent = df_other[2]
    other_sent = other_sent.reset_index(drop=True)
    get_word(other_sent, d2_A, d2_APRO, d2_V)
    
    
    return df

In [13]:
df = get_lemmat_text('text1', d_A, d_APRO, d_V, d2_A, d2_APRO, d2_V)
df.head()

,1,2,3,4,5,6
0,,,,,,
21290222,1,"[ЕГО.Местоимение],[СВАТАТЬ.Глагол],[НА.Предлог...",NaN,NaN,NaN,NaN
21290222,2,"[ОЧЕВИДНЫЙ.Прилагательное],[-.Неопределено],[Э...",NaN,NaN,NaN,NaN
21290222,3,"[НУ.Междометие],[А.Союз],[НЕВЕРОЯТНЫЙ.Прилагат...","[kw#КАНДИДАТУРА.Существительное],[NE#ВИКТОР.Им...",[kw#КАНДИДАТУРА.Существительное],"[NE#ВИКТОР.Имя.13903.True],[NE#ЗУБКОВ.Фамилия....","[NE#ВИКТОР.Имя.13903.True],[NE#ЗУБКОВ.Фамилия...."
21290222,4,"[ЕГО.Местоимение],[СВАТАТЬ.Глагол],[НА.Предлог...","[kw#ЛУЧШИЙ.Прилагательное],[NE#СОВЕТ.Существит...",[kw#ЛУЧШИЙ.Прилагательное],"[NE#СОВЕТ.Существительное.7625],[NE#ФЕДЕРАЦИЯ....",NaN
21290222,5,"[И.Союз],[УЖ.Частица],[ТЕ.Местоимение],[БОЛЕЕ....","[kw#ПРЕЕМНИК.Существительное],[kw#ПОДАТЬ.Сущес...","[kw#ПРЕЕМНИК.Существительное],[kw#ПОДАТЬ.Сущес...",NaN,NaN


In [14]:
df = get_lemmat_text('text2', d_A, d_APRO, d_V, d2_A, d2_APRO, d2_V)
df.head()

,1,2,3,4,5,6
0,,,,,,
25707643,0,"[Deutsche.Неопределено],[Post.Неопределено]",NaN,NaN,NaN,NaN
25707643,1,"[kw#ПРОКУРОР.Существительное],[ЗАНЯТЬСЯ.Глагол...","[kw#ПРОКУРОР.Существительное],[kw#ГИГАНТ.Сущес...","[kw#ПРОКУРОР.Существительное],[kw#ГИГАНТ.Сущес...",NaN,NaN
25707643,2,"[НЕМЕЦКИЙ.Прилагательное],[kw#ПРОКУРАТУРА.Суще...","[kw#ПРОКУРАТУРА.Существительное],[kw#ПОДОЗРЕВА...","[kw#ПРОКУРАТУРА.Существительное],[kw#ПОДОЗРЕВА...","[NE#Deutsche.Неопределено.21593.False],[NE#Pos...","[NE#Deutsche.Неопределено.21593.False],[NE#Pos..."
25707643,4,"[kw#ПРОКУРАТУРА.Существительное],[kw#ОБЫСКАТЬ....","[kw#ПРОКУРАТУРА.Существительное],[kw#ОБЫСКАТЬ....","[kw#ПРОКУРАТУРА.Существительное],[kw#ОБЫСКАТЬ....","[NE#Deutsche.Неопределено.21593.False],[NE#Pos...","[NE#Deutsche.Неопределено.21593.False],[NE#Pos..."
25707643,5,"[В.Предлог],[СВЯЗЬ.Существительное],[С.Предлог...",[kw#ОБЫСК.Существительное],[kw#ОБЫСК.Существительное],NaN,NaN


In [15]:
import sqlite3

In [21]:
def make_db():
    if os.path.isfile('freq.db'):
        os.remove('freq.db')
    conn = sqlite3.connect('freq.db')
    conn.execute('create table if not exists adjectives(lemma text, freq_first int, freq_other int)')
    conn.execute('create table if not exists pronouns(lemma text, freq_first int, freq_other int)')
    conn.execute('create table if not exists verbs(lemma text, freq_first int, freq_other int)')
    return conn


def proc(conn, d, d2, name):
    for lemma in d.keys() | d2.keys():
        if lemma not in d:
            d_l = 0
        else:
            d_l = d[lemma]
        if lemma not in d2:
            d2_l = 0
        else:
            d2_l = d2[lemma]
            
        conn.execute('insert into ' + name + '(lemma, freq_first, freq_other) values (?, ?, ?)',
                         (lemma, d_l, d2_l))
        conn.commit()

In [22]:
conn = make_db()
try:
    proc(conn, d_A, d2_A, 'adjectives')
    proc(conn, d_APRO, d2_APRO, 'pronouns')
    proc(conn, d_V, d2_V, 'verbs')
finally:
    conn.close()